## Using Gemini Model

In [ ]:
import os

In [ ]:
from google.colab import userdata
key = userdata.get('tejasraval607apikey')

In [ ]:
import time
from google import genai
from google.genai import types

client = genai.Client(api_key = key)

operation = client.models.generate_videos(
    model="veo-3.0-generate-preview",
    prompt="A cinematic shot of a majestic lion in the savannah.",
    config=types.GenerateVideosConfig(negative_prompt="cartoon, drawing, low quality"),
)

# Poll the operation status until the video is ready.
while not operation.done:
    print("Waiting for video generation to complete...")
    time.sleep(10)
    operation = client.operations.get(operation)

# Download the generated video.
generated_video = operation.response.generated_videos[0]
client.files.download(file=generated_video.video)
generated_video.video.save("parameters_example.mp4")
print("Generated video saved to parameters_example.mp4")

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED'}}

## **Using AIML platform Model**

In [ ]:
import requests
import time

base_url = "https://api.aimlapi.com/v2"
# Insert your AIML API Key instead of <YOUR_AIMLAPI_KEY>:

aimlapi_key = userdata.get("secretName")

# Creating and sending a video generation task to the server
def generate_video():
    url = f"{base_url}/generate/video/google/generation"
    headers = {
        "Authorization": f"Bearer {aimlapi_key}",
    }

    data = {
        "model": "google/veo3",
        "prompt": '''
A menacing evil dragon appears in a distance above the tallest mountain, then rushes toward the camera with its jaws open, revealing massive fangs. We see it's coming.
'''
    }

    response = requests.post(url, json=data, headers=headers)

    if response.status_code >= 400:
        print(f"Error: {response.status_code} - {response.text}")
    else:
        response_data = response.json()
        # print(response_data)
        return response_data


# Requesting the result of the task from the server using the generation_id
def get_video(gen_id):
    url = f"{base_url}/generate/video/google/generation"
    params = {
        "generation_id": gen_id,
    }

    headers = {
        "Authorization": f"Bearer {aimlapi_key}",
        "Content-Type": "application/json"
        }

    response = requests.get(url, params=params, headers=headers)
    # print("Generation:", response.json())
    return response.json()



def main():
     # Running video generation and getting a task id
    gen_response = generate_video()
    gen_id = gen_response.get("id")
    print("Gen_ID:  ", gen_id)

    # Trying to retrieve the video from the server every 10 sec
    if gen_id:
        start_time = time.time()

        timeout = 1000
        while time.time() - start_time < timeout:
            response_data = get_video(gen_id)

            if response_data is None:
                print("Error: No response from API")
                break

            status = response_data.get("status")
            print("Status:", status)

            if status == "waiting" or status == "active" or  status == "queued" or status == "generating":
                print("Still waiting... Checking again in 10 seconds.")
                time.sleep(10)
            else:
                print("Processing complete:/n", response_data)
                return response_data

        print("Timeout reached. Stopping.")
        return None




In [ ]:
main()

Error: 403 - {"requestId":"14f1a0cb-2c8c-4bad-9140-3a69035e5eb2","statusCode":403,"timestamp":"2025-08-23T11:58:09.698Z","path":"/v2/generate/video/google/generation","meta":{"name":"plan_rule.api_token","tag":"tier_2"},"message":"You have exhausted the available [plan_rule.api_token:tier_2] resource limit. Update your payment method to continue using the service. For more information please visit https://aimlapi.com/app/billing"}


AttributeError: 'NoneType' object has no attribute 'get'

## Using HuggingFace model

In [ ]:
from google.colab import userdata


In [ ]:
import os
from huggingface_hub import InferenceClient
key = userdata.get('HF_TOKEN')

client = InferenceClient(
    provider="auto",
    api_key=key,
)

video = client.text_to_video(
    "a man runing after a girl, a man's head is dimound head of ben 10",
    model="Wan-AI/Wan2.2-TI2V-5B",
)

if isinstance(video, (bytes, bytearray)):
    print("it's using bytes")
    with open("output.mp4", "wb") as f:
        f.write(video)
    print("Saved as output.mp4 ✅")

    from IPython.display import display, Video

# Assuming the generated video is stored in a variable named 'video'
# and it's a path to a local file or a URL
display(Video(filename="output.mp4"))

HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/replicate/v1/models/wan-video/wan-2.2-5b-fast/predictions (Request ID: Root=1-68aa16b9-07e9dafb3d58aa951dc052eb;76007e25-16ca-44af-8ea9-55202d29f4f3)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

it's using bytes
Saved as output.mp4 ✅


In [ ]:
from IPython.display import display, Video

# Assuming the generated video is stored in a variable named 'video'
# and it's a path to a local file or a URL
display(Video(filename="output.mp4"))

TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

## **RunwayML API**

## stable diffusion api

In [ ]:
# Python requests example
import requests
import json

from google.colab import userdata

stablediffusionkey = userdata.get('stablediffusionrupali')

import requests
import json

url = "https://modelslab.com/api/v6/video/text2video"

payload = json.dumps({
  "key":stablediffusionkey,
    "model_id":"cogvideox",
    "prompt":"Space Station in space",
    "negative_prompt":"low quality",
    "height":512,
    "width":512,
    "num_frames":25,
    "num_inference_steps":20,
    "guidance_scale":7,
    "upscale_height":640,
    "upscale_width":1024,
    "upscale_strength":0.6,
    "upscale_guidance_scale":12,
    "upscale_num_inference_steps":20,
    "output_type":"mp4",
    "webhook":None,
    "track_id":None
})

headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

{"status":"processing","tip":"Your image is processing in background, you can get this video using fetch API","eta":60,"message":"Try to fetch request after seconds estimated","fetch_result":"https:\/\/modelslab.com\/api\/v6\/video\/fetch\/150007663","id":150007663,"output":[],"meta":{"adapter_lora":"v3_sd15_adapter","base64":"no","clip_skip":null,"controlnet":null,"controlnet_images":null,"domain_lora_scale":1,"file_prefix":"82ca0f62-d090-4ae1-be39-aed3b63d4241","fps":7,"guidance_scale":7,"height":512,"id":"150007663","improved_sampling_seed":42,"instant_response":"no","ip_adapter_id":null,"ip_adapter_image":null,"ip_adapter_scale":0.6,"lora_models":null,"lora_strength":1,"model_id":"cogvideox","motion_lora_strength":1,"motion_loras":null,"motion_module":"v3_sd15_mm","negative_prompt":"low quality","num_frames":25,"num_inference_steps":20,"output_type":"mp4","prompt":"Space Station in space","seed":182254464,"temp":"no","track_id":null,"upscale_guidance_scale":12,"upscale_height":640,

In [ ]:
import time
import requests
from IPython.display import Video,HTML

# Your fetch result URL
fetch_url = "https://modelslab.com/api/v6/video/fetch/150007663"

video_url = None
print("Checking for video completion...")

# Poll with POST
for i in range(20):  # up to ~200s total
    r = requests.post(fetch_url, json={"key": stablediffusionkey})   # <-- POST instead of GET
    data = r.json()
    print(data)

    if "output" in data and data["output"]:
        video_url = data["output"][0]
        print("✅ Video ready:", video_url)
        break

    print(f"⏳ Status: {data.get('status')} | waiting...")
    time.sleep(15)

# Show video inline if ready
if video_url:
    display(HTML(f"""
<video width="640" height="360" controls>
  <source src="{video_url}" type="video/mp4">
  Your browser does not support the video tag.
</video>
"""))
else:
    print("Video not ready yet. Try again later.")



Checking for video completion...
{'status': 'success', 'id': 150007663, 'output': ['https://pub-3626123a908346a7a8be8d9295f44e26.r2.dev/video_generations/82ca0f62-d090-4ae1-be39-aed3b63d4241.mp4'], 'proxy_links': ['https://cdn2.stablediffusionapi.com/video_generations/82ca0f62-d090-4ae1-be39-aed3b63d4241.mp4']}
✅ Video ready: https://pub-3626123a908346a7a8be8d9295f44e26.r2.dev/video_generations/82ca0f62-d090-4ae1-be39-aed3b63d4241.mp4
